Import packages

In [1]:
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup
import csv
import time
import sys

Set URL and directory

In [21]:
p_dir = 'Downloaded_pages/2012-US-Presidential-Election/'

pages_url = []
thread_url = {}

for i in range(1,10):
    pages_url.append(f'https://debatepolitics.com/search/36136/?page={i}&q=2012+US+Presidential+Election&t=post&c[child_nodes]=1&c[nodes][0]=153&o=replies')

Scrape data and load onto dictonary

In [23]:
i = 1
for page in pages_url:
    page = requests.get(page)
    page_bs = BeautifulSoup(page.content, 'html.parser')
    disc = page_bs.find('div', {'class': 'block-container'})
    for thread in disc.find_all('h3', {'class': 'contentRow-title'}):
        url_val = 'https://debatepolitics.com' + thread.find('a').get('href')
        thread_url [thread.get_text()] = url_val
    
    sleep(2)
    print(f'Percentage of scrape complete {int((i/len(pages_url))*100)}%')
    i+=1

Percentage of scrape complete 11%
Percentage of scrape complete 22%
Percentage of scrape complete 33%
Percentage of scrape complete 44%
Percentage of scrape complete 55%
Percentage of scrape complete 66%
Percentage of scrape complete 77%
Percentage of scrape complete 88%
Percentage of scrape complete 100%


Scrape from each thread

In [24]:
len(thread_url)

44

In [25]:
i = 1
for title, url in thread_url.items():
    thread_html = requests.get(url)
    with open (p_dir + str(i) + '.html', 'w+') as f:
        f.write(str(thread_html.content))
    f.close()
    print(f'Percentage download completed {int((i+1)/len(thread_url))*100}%')
    i += 1
    sleep(2)

Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage

In [26]:
thread_data = []

for i in range(1, 45):
    with open (p_dir + str(i) + '.html', 'r') as f:
        dat = BeautifulSoup(f, 'html.parser')
        dat_com  = dat.find('div', {'class' : 'block-container lbContainer'})
        for comment in dat_com.find_all('article', {'class' : 'message message--post js-post js-inlineModContainer'}):
            comment_msg = ''.join(re.findall('[^\\\\tn]+[a-zA-Z0-9,.!@?&$%]', comment.find('div', {'class' : 'bbWrapper'}).get_text()))
            author = ''.join(re.findall('[^\\\\tn]+[a-zA-Z0-9,.!@?&$%]', comment.find('div', {'class' : 'message-userDetails'}).find('a', {'class' : 'username'}).get_text()))
            # date = comment.find('header', {'class' : 'message-attribution message-attribution--split'}).find('a').get_text('u-concealed')
            user_info = {}
            for user_extra in comment.find_all('dl', {'class' : 'pairs pairs--justified'}):
                user_info[user_extra.find('dt').get_text()] = ''.join(re.findall('[^\\\\nt]+[a-zA-Z0-9,.!@?&$%]',user_extra.find('dd').get_text()))
            # print(user_info)
            if 'Gender' in user_info and user_info['Gender'] != 'Undisclosed':
                gender = user_info['Gender']
            else:
                gender = None
            if 'Location' in user_info and user_info['Location'] != 'Undisclosed':
                location = user_info['Location']
            else:
                location = None
            if 'Political Leaning' in user_info and user_info['Political Leaning'] != 'Undisclosed':
                polLean = user_info['Political Leaning']
            else:
                polLean = None
            thread_data.append([author, comment_msg, i, gender, location, polLean])
        
        print(f'completed parsing {i}.html')
    f.close()

completed parsing 1.html
completed parsing 2.html
completed parsing 3.html
completed parsing 4.html
completed parsing 5.html
completed parsing 6.html
completed parsing 7.html
completed parsing 8.html
completed parsing 9.html
completed parsing 10.html
completed parsing 11.html
completed parsing 12.html
completed parsing 13.html
completed parsing 14.html
completed parsing 15.html
completed parsing 16.html
completed parsing 17.html
completed parsing 18.html
completed parsing 19.html
completed parsing 20.html
completed parsing 21.html
completed parsing 22.html
completed parsing 23.html
completed parsing 24.html
completed parsing 25.html
completed parsing 26.html
completed parsing 27.html
completed parsing 28.html
completed parsing 29.html
completed parsing 30.html
completed parsing 31.html
completed parsing 32.html
completed parsing 33.html
completed parsing 34.html
completed parsing 35.html
completed parsing 36.html
completed parsing 37.html
completed parsing 38.html
completed parsing 39.

In [27]:
clm_names = ['Poster' , 'Content', 'Thread', 'Gender', 'Location', 'Political_leaning']

df = pd.DataFrame(thread_data, columns = clm_names)

df.head()

,Poster,Content,Thread,Gender,Location,Political_leaning
0,lpast,cpwill saidYou claimed to have been Pro-Life.C...,1,Male,Fla,Independen
1,cpwill,lpast saidI was for the lionshare of my life u...,1,Male,USofA,Conservative
2,lpast,cpwill saidyes. so as soon as it became expen...,1,Male,Fla,Independen
3,cpwill,lpast saidLike I said...you would take the goo...,1,Male,USofA,Conservative
4,Sheik Yerbut,lpast saidLike I said...you would take the goo...,1,None,None,Liberal


In [28]:
df.to_csv('2012_US_Presidential_Elections.csv', index = False)